In [2]:
from pyspark.sql import SQLContext
from pyspark.sql.functions import *

sqlsc = SQLContext(sc)

## Create a DataFrame

In [3]:
df = sqlsc.read.format("jdbc") \
  .option("url", "jdbc:postgresql://localhost/cloudera?user=cloudera") \
  .option("dbtable", "gameclicks") \
  .load()

## DataFrames are just like tables

In [11]:
df.show(5)

+--------------------+-------+------+-------------+-----+------+---------+
|           timestamp|clickid|userid|usersessionid|ishit|teamid|teamlevel|
+--------------------+-------+------+-------------+-----+------+---------+
|2016-05-26 15:06:...|    105|  1038|         5916|    0|    25|        1|
|2016-05-26 15:07:...|    154|  1099|         5898|    0|    44|        1|
|2016-05-26 15:07:...|    229|   899|         5757|    0|    71|        1|
|2016-05-26 15:07:...|    322|  2197|         5854|    0|    99|        1|
|2016-05-26 15:07:...|     22|  1362|         5739|    0|    13|        1|
+--------------------+-------+------+-------------+-----+------+---------+
only showing top 5 rows



In [5]:
df.printSchema()

root
 |-- timestamp: timestamp (nullable = false)
 |-- clickid: integer (nullable = false)
 |-- userid: integer (nullable = false)
 |-- usersessionid: integer (nullable = false)
 |-- ishit: integer (nullable = false)
 |-- teamid: integer (nullable = false)
 |-- teamlevel: integer (nullable = false)



In [6]:
df.count()

755806

### select and filter

In [10]:
df.select(['timestamp','clickid']).show(5)

+--------------------+-------+
|           timestamp|clickid|
+--------------------+-------+
|2016-05-26 15:06:...|    105|
|2016-05-26 15:07:...|    154|
|2016-05-26 15:07:...|    229|
|2016-05-26 15:07:...|    322|
|2016-05-26 15:07:...|     22|
+--------------------+-------+
only showing top 5 rows



In [14]:
df.filter(df['userid']==1).select(['timestamp','clickid','userid']).show(5)

+--------------------+-------+------+
|           timestamp|clickid|userid|
+--------------------+-------+------+
|2016-05-26 15:19:...|    326|     1|
|2016-05-26 15:29:...|    329|     1|
|2016-05-26 16:13:...|   1253|     1|
|2016-05-26 16:18:...|   1256|     1|
|2016-05-26 16:23:...|   1258|     1|
+--------------------+-------+------+
only showing top 5 rows



### groupby

In [9]:
df.groupBy("ishit").count().show()

+-----+------+
|ishit| count|
+-----+------+
|    0|672423|
|    1| 83383|
+-----+------+



### join with other DataFrames

In [11]:
df2 = sqlsc.read.format("jdbc") \
  .option("url", "jdbc:postgresql://localhost/cloudera?user=cloudera") \
  .option("dbtable", "adclicks") \
  .load()

In [12]:
df2.printSchema()

root
 |-- timestamp: timestamp (nullable = false)
 |-- txid: integer (nullable = false)
 |-- usersessionid: integer (nullable = false)
 |-- teamid: integer (nullable = false)
 |-- userid: integer (nullable = false)
 |-- adid: integer (nullable = false)
 |-- adcategory: string (nullable = false)



In [13]:
merge = df.join(df2, 'userid')

In [14]:
merge.printSchema()

root
 |-- userid: integer (nullable = false)
 |-- timestamp: timestamp (nullable = false)
 |-- clickid: integer (nullable = false)
 |-- usersessionid: integer (nullable = false)
 |-- ishit: integer (nullable = false)
 |-- teamid: integer (nullable = false)
 |-- teamlevel: integer (nullable = false)
 |-- timestamp: timestamp (nullable = false)
 |-- txid: integer (nullable = false)
 |-- usersessionid: integer (nullable = false)
 |-- teamid: integer (nullable = false)
 |-- adid: integer (nullable = false)
 |-- adcategory: string (nullable = false)



In [15]:
merge.show(5)

+------+--------------------+-------+-------------+-----+------+---------+--------------------+-----+-------------+------+----+----------+
|userid|           timestamp|clickid|usersessionid|ishit|teamid|teamlevel|           timestamp| txid|usersessionid|teamid|adid|adcategory|
+------+--------------------+-------+-------------+-----+------+---------+--------------------+-----+-------------+------+----+----------+
|   231|2016-06-08 00:45:...| 376796|        23626|    0|   142|        4|2016-06-08 01:40:...|23669|        23626|   142|  27|     games|
|   231|2016-06-08 00:45:...| 376796|        23626|    0|   142|        4|2016-06-08 09:24:...|24122|        23626|   142|   4|     games|
|   231|2016-06-08 00:45:...| 376796|        23626|    0|   142|        4|2016-06-08 17:21:...|24659|        23626|   142|  22| computers|
|   231|2016-06-08 00:45:...| 376796|        23626|    0|   142|        4|2016-06-08 23:34:...|25076|        23626|   142|  21|    movies|
|   231|2016-06-08 00:45:..

## Running SQL Queries

In [37]:
# for Spark 2.*
# df.createOrReplaceTempView('table')
# for Spark 1.6
df.registerTempTable('sqltable')
sqlsc.sql("SELECT * FROM sqltable WHERE ishit=0 AND userid=1 AND clickid<500").show()

+--------------------+-------+------+-------------+-----+------+---------+
|           timestamp|clickid|userid|usersessionid|ishit|teamid|teamlevel|
+--------------------+-------+------+-------------+-----+------+---------+
|2016-05-26 15:19:...|    326|     1|         5835|    0|    99|        1|
|2016-05-26 15:29:...|    329|     1|         5835|    0|    99|        1|
+--------------------+-------+------+-------------+-----+------+---------+

